In [1]:

# Importing Libraries 
import cv2 
import mediapipe as mp 
from math import hypot 
import screen_brightness_control as sbc 
import numpy as np 

In [2]:
mpHands=mp.solutions.hands
hands=mpHands.Hands(static_image_mode=False,model_complexity=1,min_detection_confidence=0.75,min_tracking_confidence=0.75,max_num_hands=2)
Draw=mp.solutions.drawing_utils

In [ ]:
cap=cv2.VideoCapture(0)

while True:
    #read video by frame
    _,frame=cap.read()
    #flip image
    frame=cv2.flip(frame,1)
    #convert bgr image to rgb image
    framergb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    #process the rgb image
    Process=hands.process(framergb)

    landmarklist=[]

    #if hands are present in image(frame)
    if Process.multi_hand_landmarks:

        #detect handmarks
        for handlm in Process.multi_hand_landmarks:
            for _id,landmarks in enumerate(handlm.landmark):
                #store height,width of image
                height,width,color_channel=frame.shape

                #calculate and append x,y cordinates of
                #hand marks from image(frame) to lmlist
                x,y=int(landmarks.x*width),int(landmarks.y*height)
                landmarklist.append([_id,x,y])
            #draw landmarks
            Draw.draw_landmarks(frame,handlm,mpHands.HAND_CONNECTIONS)
    
    #if landmark list is not empty
    if landmarklist!=[]:
        #store x,y coordinates of (tip of) thump
        x_1,y_1=landmarklist[4][1],landmarklist[4][2]
        #store x,y coordinates of (tip of) index finger
        x_2,y_2=landmarklist[8][1],landmarklist[8][2]

        #draw circle on thump and index finger tip
        cv2.circle(frame,(x_1,y_1),7,(0,255,0),cv2.FILLED)
        cv2.circle(frame,(x_2,y_2),7,(0,255,0),cv2.FILLED)

        #draw line from tip of thump to tip of index finger
        cv2.line(frame,(x_1,y_1),(x_2,y_2),(0,255,0),3)

        #calculate squareroot of the sum of squares of the specified arguments
        L=hypot(x_2-x_1,y_2-y_1)

        #1-D Linear interpolant to a function with given discrete data points
        #(hand range 15-120,brightness range 0-100) evaluated at length 
        b_level=np.interp(L,[15,220],[0,100])

        #set brightness
        sbc.set_brightness(int(b_level))
    #display video and when 'q' is entered
    #destroy the windows

    cv2.imshow('image',frame)
    if cv2.waitKey(1)&0xff==ord('q'):
        break